# use firecrawl AI for scrapping data

In [101]:
from firecrawl import FirecrawlApp, ScrapeOptions
import pandas as pd
import pdfplumber
import requests
import re
import os

In [2]:
FIRECRAWL_API_KEY = "fc-d28fcfeecd9e4064ad6cc334253004bf"

In [3]:
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

In [4]:
annual_report_urls = {
"2023": "https://www.itcportal.com/about-itc/shareholder-value/annual-reports/itc-annual-report-2023/pdf/ITC-Report-and-Accounts-2023.pdf",
"2024": "https://www.itcportal.com/about-itc/shareholder-value/annual-reports/itc-annual-report-2024/pdf/ITC-Report-and-Accounts-2024.pdf"
}

In [5]:
def extract_financials_from_pdf(url_1, year):
    response = app.scrape_url(url_1, formats=['html'])
    
    full_text = response.html
    
    cleaned_text = re.sub(r'\d+(\.\d+)?%', '', full_text)
    
    revenue = None
    net_profit = None
    
    rev_pattern = re.search(r"(Revenue from Operations|Gross Revenue)[^\d₹]{0,20}(₹?\s?[\d,]+.\d+|\d{2,})", cleaned_text, re.IGNORECASE)
    
    
    profit_pattern = re.search(r"(Net Profit|Profit After Tax)[^\d₹]{0,20}(₹?\s?[\d,]+.\d+|\d{2,})", cleaned_text, re.IGNORECASE)
    
    
    if rev_pattern:
        revenue = rev_pattern.group(2)
    if profit_pattern:
        net_profit = profit_pattern.group(2)
        
    return {"Year": year, "Revenue(₹cr)": revenue or "Not found", "Net Profit(₹cr)": net_profit or "Not found"}

In [6]:
financials = []

for year, url in annual_report_urls.items():
    
    data = extract_financials_from_pdf(url, year)
    
    financials.append(data)
    
print(financials)

[{'Year': '2023', 'Revenue(₹cr)': '69,481', 'Net Profit(₹cr)': '18753.31'}, {'Year': '2024', 'Revenue(₹cr)': '69,446', 'Net Profit(₹cr)': '20421.97'}]


In [7]:
import yfinance as yf

C:\Users\santosh\anaconda3\lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.30.2 at yfinance/pricing.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [8]:
def get_stock_data_yahoo(ticker="ITC.NS", start="2023-01-01", end="2025-05-05"):
    
    stock_data = yf.download(ticker, start=start, end=end)
    stock_data.reset_index(inplace=True)
    
    return stock_data[['Date', 'Open', 'Close']]

In [9]:
stock_prices_df = get_stock_data_yahoo()
stock_prices_df 

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Date,Open,Close
Ticker,,ITC.NS,ITC.NS
0,2023-01-02,308.648706,310.467590
1,2023-01-03,309.208393,308.835297
2,2023-01-04,309.534824,305.010956
3,2023-01-05,306.550010,311.027252
4,2023-01-06,312.193181,312.519623
...,...,...,...
568,2025-04-25,430.049988,427.799988
569,2025-04-28,426.649994,428.799988
570,2025-04-29,428.899994,426.000000


# Store scraped data in a SQLite/PostgreSQL database

In [10]:
import sqlite3
import re

In [11]:
conn = sqlite3.connect("itc_financials.db")
cursor = conn.cursor()

In [12]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS annual_financials (
Year TEXT PRIMARY KEY,
Revenue TEXT,
Net_Profit TEXT
)
""")

In [13]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS stock_prices (
Date TEXT,
Open REAL,
Close REAL,
PRIMARY KEY (Date)
)
""")
conn.commit()

In [14]:
for item in financials:
    cursor.execute("""INSERT OR REPLACE INTO annual_financials (Year, Revenue, Net_Profit )VALUES (?, ?, ?)"""
                   , (item["Year"], item["Revenue(₹cr)"], item["Net Profit(₹cr)"]))

conn.commit()

In [15]:
stock_prices_df = stock_prices_df.reset_index()
stock_prices_df["Date"] = pd.to_datetime(stock_prices_df["Date"]).dt.strftime('%Y-%m-%d')

In [16]:
print(stock_prices_df.columns)

MultiIndex([('index',       ''),
            ( 'Date',       ''),
            ( 'Open', 'ITC.NS'),
            ('Close', 'ITC.NS')],
           names=['Price', 'Ticker'])


In [17]:
stock_prices_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in stock_prices_df.columns.values]

In [18]:
stock_prices_df.rename(columns={
'Date_': 'Date',
'Open_ITC.NS': 'Open',
'Close_ITC.NS': 'Close'
}, inplace=True)

In [19]:
print(stock_prices_df.columns)

Index(['index', 'Date', 'Open', 'Close'], dtype='object')


In [20]:
for _, row in stock_prices_df.iterrows():
    
    date_str = str(row["Date"])
    open_val = float(row["Open"]) if not pd.isnull(row["Open"]) else None
    close_val = float(row["Close"]) if not pd.isnull(row["Close"]) else None
    
    cursor.execute("""INSERT OR REPLACE INTO stock_prices (Date, Open, Close)VALUES (?, ?, ?)""",
                   (date_str, open_val, close_val))
    
conn.commit()

In [21]:
print(stock_prices_df.columns)

Index(['index', 'Date', 'Open', 'Close'], dtype='object')


In [22]:
stock_prices_df.rename(columns={
'Date_': 'Date',
'Open_ITC.NS': 'Open',
'Close_ITC.NS': 'Close'
}, inplace=True)

In [23]:
stock_prices_df

,index,Date,Open,Close
0,0,2023-01-02,308.648706,310.467590
1,1,2023-01-03,309.208393,308.835297
2,2,2023-01-04,309.534824,305.010956
3,3,2023-01-05,306.550010,311.027252
4,4,2023-01-06,312.193181,312.519623
...,...,...,...,...
568,568,2025-04-25,430.049988,427.799988
569,569,2025-04-28,426.649994,428.799988
570,570,2025-04-29,428.899994,426.000000
571,571,2025-04-30,427.399994,425.799988


In [24]:
conn = sqlite3.connect("financial_data.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS stock_prices (
Date TEXT PRIMARY KEY,
Open REAL,
Close REAL
)
""")

In [25]:
for _, row in stock_prices_df.iterrows():
    
    open_val = float(row["Open"]) if pd.notnull(row["Open"]) else None
    close_val = float(row["Close"]) if pd.notnull(row["Close"]) else None
    
    cursor.execute("""INSERT OR REPLACE INTO stock_prices (Date, Open, Close)VALUES (?, ?, ?)""",
                   (row["Date"], open_val, close_val))
    
conn.commit()

In [26]:
pd.read_sql("SELECT * FROM annual_financials", conn)

,Year,Revenue,Net_Profit
0,2023,"69,481",18753.31
1,2024,"69,446",20421.97


In [27]:
pd.read_sql("SELECT * FROM stock_prices LIMIT 5", conn)

,Date,Open,Close
0,2023-01-02,308.648706,310.467590
1,2023-01-03,309.208393,308.835297
2,2023-01-04,309.534824,305.010956
3,2023-01-05,306.550010,311.027252
4,2023-01-06,312.193181,312.519623


In [28]:
conn.close()

In [29]:
conn = sqlite3.connect("financial_data.db")
cursor = conn.cursor()

# Convert data into embeddings

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import json

C:\Users\santosh\anaconda3\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [31]:
annual_financials_df = pd.read_sql("SELECT * FROM annual_financials", conn)
stock_prices_df = pd.read_sql("SELECT * FROM stock_prices", conn)

In [32]:
text_data = stock_prices_df["Date"].astype(str).tolist()

In [33]:
vectorizer = TfidfVectorizer(max_features=10)  
tfidf_matrix = vectorizer.fit_transform(text_data)

In [34]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df["Date"] = stock_prices_df["Date"] 

In [35]:
tfidf_df.to_sql("tfidf_embeddings", conn, if_exists="replace", index=False)

573

In [36]:
print(tfidf_df.head())

         01        02        03        04        05        06   10      2023  \
0  0.641836  0.652732  0.000000  0.000000  0.000000  0.000000  0.0  0.402477   
1  0.641836  0.000000  0.652732  0.000000  0.000000  0.000000  0.0  0.402477   
2  0.637043  0.000000  0.000000  0.659241  0.000000  0.000000  0.0  0.399472   
3  0.623958  0.000000  0.000000  0.000000  0.676451  0.000000  0.0  0.391267   
4  0.618211  0.000000  0.000000  0.000000  0.000000  0.683764  0.0  0.387663   

   2024  2025        Date  
0   0.0   0.0  2023-01-02  
1   0.0   0.0  2023-01-03  
2   0.0   0.0  2023-01-04  
3   0.0   0.0  2023-01-05  
4   0.0   0.0  2023-01-06  


In [37]:
conn.close()